In [413]:


from urllib.request import urlopen

In [414]:
import json

with open('mentalhealth.json','r',encoding='utf-8') as _mh_rd:
    _mh_json = json.load(_mh_rd)

In [415]:
list(_mh_json.keys())

['meta', 'data']

In [416]:
#How many records did we get?
len(_mh_json['data'])

8745

In [417]:
#So we have over 8000 records of data. We will now push this into our MongoDB to store.
from pymongo import MongoClient
try:
    client = MongoClient(host = '192.168.56.30:27017')
    client.server_info()
    #Fix below code for a successful connection
except Exception as e:
    print(e)
else:    
    print("Connection successful")

Connection successful


In [418]:
#We will create our database in MongoDB named []
db = client['mentalhealth']

#add a collection to our database
mh_collection = db.mentalhealth

#and insert our data
insert_mh= mh_collection.insert_one(_mh_json)

In [419]:
#To organize our data into key:values, we will get the heads of our
#columns from the metadata part of the dictionary we loaded

headers= []
for i in range(len(_mh_json['meta']['view']['columns'])):
    headers.append(_mh_json['meta']['view']['columns'][i]['name'])

print(headers)

['sid', 'id', 'position', 'created_at', 'created_meta', 'updated_at', 'updated_meta', 'meta', 'Service Year', 'OMH Region Code', 'OMH Region Label', 'County Label', 'Age Group', 'Rate Code Group', 'Medicaid Enrollee Count by County', 'Count of Recipients by Rate Code Group and County', 'Units Total', 'Paid Claim Total']


In [420]:
#and get our value data as follows
body = []
for k in mh_collection.data.find({},{"data":1}):
    temprow = []
    temprow.append(k.values())
body.append(temprow)

print(type(body))
print(len(body))


<class 'list'>
1


In [421]:
tmp=[]
for i in body[0][0]:
    #print("-----")
    #print(i)
    tmp.append(i)

print(tmp[1])


#for s in k:
 #   print(len(s))

[['row-fhi8.adwm-hd87', '00000000-0000-0000-7863-CE3E01AF952E', 0, 1578688714, None, 1578688714, None, '{ }', '2014', '2', 'Central NY', 'Broome', 'Adult', 'Assertive Community Treatment', '27255', '61', '622', '899264'], ['row-h6d4.cw2r-4gbf', '00000000-0000-0000-502E-C019510D5D14', 0, 1578688714, None, 1578688714, None, '{ }', '2014', '2', 'Central NY', 'Broome', 'Adult', 'OMH Licensed Clinic Treatment', '27255', '2329', '16957', '1764794'], ['row-3acn_meab~t3wx', '00000000-0000-0000-396D-F7C293AF28E1', 0, 1578688714, None, 1578688714, None, '{ }', '2014', '2', 'Central NY', 'Broome', 'Adult', 'Community Residence', '27255', '92', '610', '1437842'], ['row-i62b.ep7j_9txk', '00000000-0000-0000-FA80-0F21C8C01497', 0, 1578688714, None, 1578688714, None, '{ }', '2014', '2', 'Central NY', 'Broome', 'Adult', 'Comprehensive Psychiatric Emergency Program\nNon-EOB', '27255', '179', '246', '154516'], ['row-t9by~9qic~d7fr', '00000000-0000-0000-2B74-2A501DB89796', 0, 1578688714, None, 1578688714,

In [422]:
#let's flatten our list!
mhbody = [item1 for item1 in tmp[1]]
print(len(mhbody))
mhbody[:2]


8745


[['row-fhi8.adwm-hd87',
  '00000000-0000-0000-7863-CE3E01AF952E',
  0,
  1578688714,
  None,
  1578688714,
  None,
  '{ }',
  '2014',
  '2',
  'Central NY',
  'Broome',
  'Adult',
  'Assertive Community Treatment',
  '27255',
  '61',
  '622',
  '899264'],
 ['row-h6d4.cw2r-4gbf',
  '00000000-0000-0000-502E-C019510D5D14',
  0,
  1578688714,
  None,
  1578688714,
  None,
  '{ }',
  '2014',
  '2',
  'Central NY',
  'Broome',
  'Adult',
  'OMH Licensed Clinic Treatment',
  '27255',
  '2329',
  '16957',
  '1764794']]

In [423]:
import pandas as pd

#We can now build a dataframe by combining our headers and their values
mhdf = pd.DataFrame(mhbody, columns=headers)
mhdf.tail()


,sid,id,position,created_at,created_meta,updated_at,updated_meta,meta,Service Year,OMH Region Code,OMH Region Label,County Label,Age Group,Rate Code Group,Medicaid Enrollee Count by County,Count of Recipients by Rate Code Group and County,Units Total,Paid Claim Total
8740,row-n7gf_zn69-fwmc,00000000-0000-0000-9D84-0363C4145B4F,0,1578688714,None,1578688714,None,{ },2018,1,Western NY,Yates,Child,OMH/OASAS Certified Community Behavioral Healt...,2066,0,0,0
8741,row-gyd4.x3bj_qxuv,00000000-0000-0000-2FA7-123196841C3A,0,1578688714,None,1578688714,None,{ },2018,1,Western NY,Yates,Child,Psych Inpatient 28,2066,0,0,0
8742,row-p4rt~m23c_cpgc,00000000-0000-0000-BC14-CC1E97F4FB89,0,1578688714,None,1578688714,None,{ },2018,1,Western NY,Yates,Child,Psych Inpatient 31,2066,0,0,0
8743,row-vhsj-wjma.s7yq,00000000-0000-0000-B05E-A7E88AEAE4D7,0,1578688714,None,1578688714,None,{ },2018,1,Western NY,Yates,Child,Psych Inpatient SPC,2066,0,0,0
8744,row-tpcu_vjir.ek5p,00000000-0000-0000-0B22-BF2C9CCA1887,0,1578688714,None,1578688714,None,{ },2018,1,Western NY,Yates,Senior,OMH Licensed Clinic Treatment,559,0,0,0


In [424]:
#We will now venture into Data Preprocessing where we will clean and polish the data before we begin analysis and visualization. 
#The following steps will be carried out in this phase;
#1. Treat missing rows, columns and values
#2. Inconsistent values like mistyped values or wrong data types
#3. Duplicate values
#4. Dropping unncessary rows

In [425]:
#We will begin by exploring our data

mhdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8745 entries, 0 to 8744
Data columns (total 18 columns):
 #   Column                                             Non-Null Count  Dtype 
---  ------                                             --------------  ----- 
 0   sid                                                8745 non-null   object
 1   id                                                 8745 non-null   object
 2   position                                           8745 non-null   int64 
 3   created_at                                         8745 non-null   int64 
 4   created_meta                                       0 non-null      object
 5   updated_at                                         8745 non-null   int64 
 6   updated_meta                                       0 non-null      object
 7   meta                                               8745 non-null   object
 8   Service Year                                       8745 non-null   object
 9   OMH Region Code    

In [426]:
#From the meta-data, we know that some of the columns here are not useful
#to our analysis and we will drop them to focus our data. The following
#columns will be dropped.

col_drop = ['sid', 'id', 'position', 'created_at', 'created_meta', 'updated_at', 'updated_meta', 'meta']

mhdf = mhdf.drop(col_drop,axis=1)



In [427]:
#We will later conduct a unique analysis of the columns and will drop rows so we won't address rows at this stage
#We will now check for NaN values in our data
mhdf[mhdf.isna().any(axis=1)]

,Service Year,OMH Region Code,OMH Region Label,County Label,Age Group,Rate Code Group,Medicaid Enrollee Count by County,Count of Recipients by Rate Code Group and County,Units Total,Paid Claim Total


In [428]:
#Our data does not have missing values

#So we will check the columns for duplicate or misspelt data

mhdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8745 entries, 0 to 8744
Data columns (total 10 columns):
 #   Column                                             Non-Null Count  Dtype 
---  ------                                             --------------  ----- 
 0   Service Year                                       8745 non-null   object
 1   OMH Region Code                                    8745 non-null   object
 2   OMH Region Label                                   8745 non-null   object
 3   County Label                                       8745 non-null   object
 4   Age Group                                          8745 non-null   object
 5   Rate Code Group                                    8745 non-null   object
 6   Medicaid Enrollee Count by County                  8745 non-null   object
 7   Count of Recipients by Rate Code Group and County  8745 non-null   object
 8   Units Total                                        8745 non-null   object
 9   Paid Claim Total   

In [429]:
#We will convert some columns to category datatype

mhdf = mhdf.astype({'OMH Region Code':'category','OMH Region Label':'category','County Label':'category','Age Group':'category','Rate Code Group':'category'})


In [430]:
#We will start by first getting the category columns in a list we can iterate the unique function on
cat_cols = mhdf.select_dtypes(include=["category"])
cat_cols = list(cat_cols.columns)
print(cat_cols)


['OMH Region Code', 'OMH Region Label', 'County Label', 'Age Group', 'Rate Code Group']


In [431]:
#To begin, we will convert all values to uppercase
for col in cat_cols:
    mhdf[col] = mhdf[col].str.upper()
    

In [432]:
for col in cat_cols:
    print("Col :", col)
    print("unique values are :",mhdf[col].sort_values().unique())
    print("________________")

Col : OMH Region Code
unique values are : ['1' '2' '3' '4' '5']
________________
Col : OMH Region Label
unique values are : ['CENTRAL NY' 'HUDSON RIVER' 'LONG ISLAND' 'NEW YORK CITY' 'WESTERN NY']
________________
Col : County Label
unique values are : ['ALBANY' 'ALLEGANY' 'BRONX' 'BROOME' 'CATTARAUGUS' 'CAYUGA' 'CHAUTAUQUA'
 'CHEMUNG' 'CHENANGO' 'CLINTON' 'COLUMBIA' 'CORTLAND' 'DELAWARE'
 'DUTCHESS' 'ERIE' 'ESSEX' 'FRANKLIN' 'FULTON' 'GENESEE' 'GREENE'
 'HAMILTON' 'HERKIMER' 'JEFFERSON' 'KINGS' 'LEWIS' 'LIVINGSTON' 'MADISON'
 'MONROE' 'MONTGOMERY' 'NASSAU' 'NEW YORK' 'NIAGARA' 'ONEIDA' 'ONONDAGA'
 'ONTARIO' 'ORANGE' 'ORLEANS' 'OSWEGO' 'OTSEGO' 'PUTNAM' 'QUEENS'
 'RENSSELAER' 'RICHMOND' 'ROCKLAND' 'SAINT LAWRENCE' 'SARATOGA'
 'SCHENECTADY' 'SCHOHARIE' 'SCHUYLER' 'SENECA' 'STEUBEN' 'SUFFOLK'
 'SULLIVAN' 'TIOGA' 'TOMPKINS' 'ULSTER' 'WARREN' 'WASHINGTON' 'WAYNE'
 'WESTCHESTER' 'WYOMING' 'YATES']
________________
Col : Age Group
unique values are : ['ADULT' 'CHILD' 'SENIOR']
______________

In [433]:
#We do not have misspelt values as well. We will continue our preprocessing
mhdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8745 entries, 0 to 8744
Data columns (total 10 columns):
 #   Column                                             Non-Null Count  Dtype 
---  ------                                             --------------  ----- 
 0   Service Year                                       8745 non-null   object
 1   OMH Region Code                                    8745 non-null   object
 2   OMH Region Label                                   8745 non-null   object
 3   County Label                                       8745 non-null   object
 4   Age Group                                          8745 non-null   object
 5   Rate Code Group                                    8745 non-null   object
 6   Medicaid Enrollee Count by County                  8745 non-null   object
 7   Count of Recipients by Rate Code Group and County  8745 non-null   object
 8   Units Total                                        8745 non-null   object
 9   Paid Claim Total   

In [434]:
#We have no missing values, our category values are ok and our columns are in the correct datatypes for analysis. 
#We are now ready to visualize our data

#PS: We choose not to change the date columns at the moment, 
#preferring to leave them as strings till they need to be worked on
mhdf.head()

,Service Year,OMH Region Code,OMH Region Label,County Label,Age Group,Rate Code Group,Medicaid Enrollee Count by County,Count of Recipients by Rate Code Group and County,Units Total,Paid Claim Total
0,2014,2,CENTRAL NY,BROOME,ADULT,ASSERTIVE COMMUNITY TREATMENT,27255,61,622,899264
1,2014,2,CENTRAL NY,BROOME,ADULT,OMH LICENSED CLINIC TREATMENT,27255,2329,16957,1764794
2,2014,2,CENTRAL NY,BROOME,ADULT,COMMUNITY RESIDENCE,27255,92,610,1437842
3,2014,2,CENTRAL NY,BROOME,ADULT,COMPREHENSIVE PSYCHIATRIC EMERGENCY PROGRAM\nN...,27255,179,246,154516
4,2014,2,CENTRAL NY,BROOME,ADULT,MH EMERGENCY,27255,1005,1539,427157


In [435]:
#We will now insert our clean data to Postgres
#We will first establish a connection with our Postgre server and create a database

import psycopg2

try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "192.168.56.30",
        port = "5432",
        database = "postgres")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute('CREATE DATABASE dapgroupm;')
    dbCursor.close()
except (Exception, psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection): 
        dbConnection.close()
        

Error while connecting to PostgreSQL database "dapgroupm" already exists



In [465]:
try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "192.168.56.30",
        port = "5432",
        database = "dapgroupm")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute("DROP TABLE newyorkmentalhealth")
    dbCursor.close()
    print("Successful")
except (Exception, psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL, Error is: ", dbError)
finally:
    if(dbConnection): 
        dbConnection.close()

Successful


In [466]:
createString = """
CREATE TABLE newyorkmentalhealth(
service_year varchar(255),
omh_region_code varchar(255),
omh_region_label varchar(255),
county_label varchar(255),
age_group varchar(255),
rate_code_group varchar(255),
med_e_c varchar(255),
crrcgc varchar(255),
units_total varchar(255),
paid_claim_total varchar(255)
);
"""
try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "192.168.56.30",
        port = "5432",
        database = "dapgroupm")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute(createString)
    dbCursor.close()
    
except (Exception, psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL, Error is: ", dbError)
else:
    print("Table created successfully")
finally:
    if(dbConnection): 
        dbConnection.close()

Table created successfully


In [438]:
mhdf.to_csv('newyorkmentalhealth.csv', index=False, na_rep='Unknown')

In [467]:
import csv


try:
    dbConnection = psycopg2.connect(
    user = "dap",
    password = "dap",
    host = "192.168.56.30",
    port = "5432",
    database = "dapgroupm")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    insertstring = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
    file = open("newyorkmentalhealth.csv", "r")
    #Inserting data from merged CSV into table in Postgres
    with dbConnection.cursor() as cur:
        #Avoiding uploading duplicate data!
        cur.execute("truncate " + "newyorkmentalhealth" + ";")
        cur.copy_expert(sql=insertstring % "newyorkmentalhealth", file=file)
        dbConnection.commit()       
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
else:
    print("Commit successful")
    print("Import is successful")
finally:
    if(dbConnection): dbConnection.close()


Commit successful
Import is successful


In [470]:
import pandas.io.sql as sqlio

try:
    dbConnection = psycopg2.connect(
    user = "dap",
    password = "dap",
    host = "192.168.56.30",
    port = "5432",
    database = "dapgroupm")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    sqlquery = """SELECT * FROM newyorkmentalhealth;"""
    mentalhealth_query = sqlio.read_sql_query(sqlquery, dbConnection)
    mentalhealth_df = pd.DataFrame().from_dict(mentalhealth_query)
    mentalhealth_df
    
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
else:
    print("Import query is successful")
    mentalhealth_df.info()
finally:
    if(dbConnection): 
        dbConnection.close()


Import query is successful


In [471]:
mentalhealth_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8745 entries, 0 to 8744
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   service_year      8745 non-null   object
 1   omh_region_code   8745 non-null   object
 2   omh_region_label  8745 non-null   object
 3   county_label      8745 non-null   object
 4   age_group         8745 non-null   object
 5   rate_code_group   8745 non-null   object
 6   med_e_c           8745 non-null   object
 7   crrcgc            8745 non-null   object
 8   units_total       8745 non-null   object
 9   paid_claim_total  8745 non-null   object
dtypes: object(10)
memory usage: 683.3+ KB


In [472]:
mentalhealth_df.head()

,service_year,omh_region_code,omh_region_label,county_label,age_group,rate_code_group,med_e_c,crrcgc,units_total,paid_claim_total
0,2014,2,CENTRAL NY,BROOME,ADULT,ASSERTIVE COMMUNITY TREATMENT,27255,61,622,899264
1,2014,2,CENTRAL NY,BROOME,ADULT,OMH LICENSED CLINIC TREATMENT,27255,2329,16957,1764794
2,2014,2,CENTRAL NY,BROOME,ADULT,COMMUNITY RESIDENCE,27255,92,610,1437842
3,2014,2,CENTRAL NY,BROOME,ADULT,COMPREHENSIVE PSYCHIATRIC EMERGENCY PROGRAM\nN...,27255,179,246,154516
4,2014,2,CENTRAL NY,BROOME,ADULT,MH EMERGENCY,27255,1005,1539,427157


In [473]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.barplot(x="county_label", y="units_total", data=mentalhealth_df)

TypeError: Neither the `x` nor `y` variable appears to be numeric.